# Machine Learning Preparation  
**Goal: to prepare data for machine learning by concatenating features (team attributes) with output (team score) and to standardize the data**  

In [1]:
#load packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [5]:
#load the points
points = pd.read_csv('~/Documents/NIH/BIOF509/Project/Scikit-learn-Project/premier_league_scores.csv', index_col = 0)
points.head()

,Score,Season,Team
0,86,2009/2010,Chelsea
1,85,2009/2010,Manchester United
2,75,2009/2010,Arsenal
3,70,2009/2010,Tottenham Hotspur
4,67,2009/2010,Manchester City


In [11]:
#load the Premier Leauge info
team_info = pd.read_csv('Premier.csv')
team_info.head()

,index,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,25,3457,10260,11,Manchester United,MUN
1,26,3458,10261,13,Newcastle United,NEW
2,27,3459,9825,1,Arsenal,ARS
3,28,3460,8659,109,West Bromwich Albion,WBA
4,29,3461,8472,106,Sunderland,SUN


In [120]:
#create a list of the dates that are in the names team attributes .csv files so that we can easily loop through files
dates = ['2010-02-22.csv', '2011-02-22.csv', '2012-02-22.csv', '2013-09-20.csv', '2014-09-19.csv', '2015-09-10.csv']

#get the seasons in a list
seasons = [i for i in points.Season.unique()]

#we are going to store our datasets in this list, each entry will be a DataFrame from a specific season
datasets = []


#loop through the filenames
for i in range(0,6):
    #get the file name
    filename = 'Data_Processing/Team_Attributes_' + dates[i]
    
    #import the data associated with filename
    data = pd.read_csv(filename, index_col = 0)
    
    #get a copy of this DataFrame
    original_df = data.copy()
    
    #now get the scores associated wtih this season
    score = points.loc[points.Season == seasons[i]].copy()

    #we are going to assemble our new DataFrame in this blank dataset
    new_df = pd.DataFrame()

    #loop through every team in the score tables for the specific season
    #we'll find the row in the original dataset that has the team name, then we'll add that row to our new dataset
    for team in score.Team:
        #a little exception handling here...
        if team == 'AFC Bournemouth':
            team = 'Bournemouth'
        else:
            pass
        
        #get the api_id of the team from our team_info DataFrame
        api_id = int(team_info.loc[team_info.team_long_name == team, 'team_api_id'])
        
        #now we'll find the row in the original DataFrame that corresponds to this api_id
        row = original_df.loc[original_df.team_api_id == api_id]
        
        #we'll add that row to our new dataframe
        new_df = new_df.append(row)
        
        #we're going to reset its associated index -- this will allow us to merge columns from different DataFrames
        new_df.reset_index()

    #drop the index and id columns
    new_df = new_df.drop(['index', 'id'], axis = 1)    
    
    #rest the index so that it runs from 0 - 19
    new_df = new_df.set_index(score.index - score.index.min())
    
    #do the same thing for the team scores for this season (i.e., a slice of the original score DataFrame)
    score = score.set_index(score.index - score.index.min())

    #our final result is going to be the result of merging the following columns from different DataFrames:
    #the team name, the season, the new data, and the score
    final = pd.concat([score.Team, score.Season, new_df, score.Score], axis = 1)
    
    #append this result to our list of DataFrames
    datasets.append(final)

In [121]:
#now we're going to standardize the numerical data, which is a subset of the total data

#loop through each of our DataFrames
for i in range(len(datasets)):
    
    #get a copy of a specific DataFrame
    data = datasets[i].copy()
    
    #get the list of columns - everything to the left of the 'date' column needs to be standardized
    columns = list(test.columns)
    
    #find where the numerical data starts
    start = columns.index('buildUpPlaySpeed')
    
    #find where the numerical data ends
    end = columns.index('date')
    
    #now we will store all of our numerical data in a matrix X
    X = data.iloc[:,start:end].copy()
    
    #we apply the Standard Scaler to the data
    new_data = StandardScaler().fit_transform(X)
    
    #we return this data to our DataFrame
    data.iloc[:,start:end] = new_data
    
    #drop the following columns since we don't need them anymore
    final = data.drop(['team_fifa_api_id', 'team_api_id', 'date'], axis = 1)
    
    #return this newly modified DataFrame to the original list of DataFrames
    datasets[i] = final
    

In [125]:
#now we'll write our datasets to new files
for i in range(len(seasons)):
    #get the dates of the corresponding season
    dates = seasons[i].split('/')
    
    #create the file name
    filename = 'team_attributes_processed-' + dates[0] + '-' + dates[1] + '.csv'
    
    #write the file
    datasets[i].to_csv(filename)
    
    #print the filename that was just written
    print('wrote the file', filename)

wrote the file team_attributes_processed-2009-2010.csv
wrote the file team_attributes_processed-2010-2011.csv
wrote the file team_attributes_processed-2011-2012.csv
wrote the file team_attributes_processed-2013-2014.csv
wrote the file team_attributes_processed-2014-2015.csv
wrote the file team_attributes_processed-2015-2016.csv


,Team,Season,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,...,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressureClass,defenceTeamWidthClass,defenceAggressionClass_Press,defenceAggressionClass_Double,defenceAggressionClass_Contain,defenceDefenderLineClass_Cover,defenceDefenderLineClass_Offside Trap,Score
0,Chelsea,2009/2010,0.838948,0.0,0.040464,-0.322535,0.345547,1.288855,-1.011775,-0.275589,...,3,0,1,2,1,0,0,1,0,86
1,Manchester United,2009/2010,0.838948,0.0,-1.063111,-1.187872,0.345547,0.802495,-0.275939,-1.194220,...,2,0,2,2,1,0,0,1,0,85
2,Arsenal,2009/2010,0.205780,0.0,-2.166686,-2.367878,-3.973795,-2.115668,-1.011775,-2.112850,...,2,0,1,2,1,0,0,1,0,75
3,Tottenham Hotspur,2009/2010,0.838948,0.0,-0.695252,-0.794537,-0.518321,-0.656587,0.459898,-1.194220,...,2,1,2,2,1,0,0,1,0,70
4,Manchester City,2009/2010,0.838948,0.0,0.040464,-0.401202,0.345547,1.288855,0.091980,-0.734904,...,3,1,2,2,1,0,0,1,0,67
5,Aston Villa,2009/2010,0.838948,0.0,-0.033107,0.385468,0.345547,-0.656587,-1.011775,0.643041,...,2,0,1,1,0,1,0,1,0,64
6,Liverpool,2009/2010,-2.326893,0.0,-1.798828,-0.794537,-1.382189,1.288855,-0.275939,-0.275589,...,3,1,2,2,1,0,0,1,0,63
7,Everton,2009/2010,-0.743973,0.0,0.408323,-0.007867,0.345547,-1.142947,-0.275939,0.643041,...,2,1,2,2,0,1,0,1,0,61
8,Birmingham City,2009/2010,0.838948,0.0,0.776181,0.778804,0.345547,1.288855,1.931570,0.643041,...,3,1,3,3,0,1,0,1,0,50
9,Blackburn Rovers,2009/2010,-1.535433,0.0,0.776181,-0.007867,0.345547,-0.170226,0.091980,0.643041,...,2,1,2,2,0,1,0,1,0,50
